In [19]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import time
import injections as inj
import functions as func
import pycbc
from scipy.interpolate import griddata
from scipy.stats import binned_statistic_2d
from pycbc import psd, detector
import lal
import seaborn as sns
from matplotlib.colors import BoundaryNorm
from matplotlib.ticker import MaxNLocator
import pandas as pd
import argparse as ap
from matplotlib.patches import Rectangle
import os
lal.ClobberDebugLevel(0)
import importlib
importlib.reload(func)
importlib.reload(inj)

<module 'injections' from '/work/rahul.dhurkunde/HM_and_precession/libraries/injections.py'>

In [15]:
def read_injections(nsets, nsignals, f_min, inj_dir):
    sg = []
    for k in range(nsets):
        filename = '%s/%s.hdf' %(inj_dir, k)
        #print('loaded', filename)
        sg = sg + inj.read_injections_HDF(filename, nsignals, f_min)

    sg_m1 = [x.m1 for x in sg]
    sg_m2 = [x.m2 for x in sg]
    mtotal = sg_m1 + sg_m2
    q = np.divide(sg_m1, sg_m2)
    end = time.time()
    return sg, sg_m1, sg_m2, mtotal, q

def read_ind_files(filename):
    f = h5py.File(filename)
    injInd = f['ind']
    return injInd

def read_SRF(filename):
    f = h5py.File(filename)
    SRF = f['SRF']
    xedges = f['xbins']
    yedges = f['ybins']
    return SRF, xedges, yedges

def read_FFs(inj_type, nsets):
    FF = np.array([])
    m1 = np.array([])
    m2 = np.array([])
    for k in range(nsets):
        filename = "/work/rahul.dhurkunde/HM_and_precession/scripts/ZDHP/%s/output/H1-FF_%s-0-10.hdf" %(inj_type, k)
        f = h5py.File(filename, 'r')

        FF = np.concatenate([FF, np.array(f['FF'])])
        m1 = np.concatenate([m1, np.array(f['mass1'])])
        m2 = np.concatenate([m2, np.array(f['mass2'])])
    return FF

def get_FF_inside_bin(binindex, FF):
    indices_file = "%s.hdf" %binindex
    injInd = read_ind_files(indices_file)
    ff_list = FF[injInd]
    return ff_list

def plot_signals(sg, injInd):
    for k in range(len(injInd)):
        ind = injInd[k]
        plt.scatter(sg[ind].m1, sg[ind].m2)

def plot_bin_vertices(x, y, xedges, yedges):
    plt.plot(xedges[x], yedges[y], 'x', color='r')
    plt.plot(xedges[x], yedges[y+1], 'x', color='r')
    plt.plot(xedges[x+1], yedges[y], 'x', color='r')
    plt.plot(xedges[x+1], yedges[y+1], 'x', color='r')

def visual_inspection_injInside(xedge, yedge, nbinsx, nbinsy, Nbins, sg, sg_m1, sg_m2):
    for n in range(Nbins):
        x = int(n%nbinsx)
        y = int((n-x)/nbinsx)
        print(xedge[x], yedge[y], min(sg_m1), min(sg_m2))
        plot_bin_vertices(x, y, xedge, yedge)

        indices_file = "%s.hdf" %n
        injInd = read_ind_files(indices_file)
        plot_signals(sg, injInd)

        plt.xlim([min(sg_m1), max(sg_m1)])
        plt.ylim([min(sg_m2), max(sg_m2)])
        plt.show()

def compute_SRF_for_bin(binindex, sg, ff_list, sigmasq_list):
    indices_file = "%s.hdf" %binindex
    injInd = read_ind_files(indices_file)
    numerator = 0.0
    denominator = 0.0
    if(len(injInd) != 0):
        for k in range(len(injInd)):
            inj_ind = injInd[k]
            numerator += sigmasq_list[k]**3 * ff_list[k]**3
            denominator += sigmasq_list[k]**3
        srf_manual = numerator/denominator
    else:
        print('No signals is bin', binindex)
        srf_manual = np.nan
    return srf_manual    

def get_signal(sg, delta_f, f_min, approximant, HMs, detect):
    if HMs==True:
        sp, sc = func.generate_signal(sg, delta_f, f_min, approximant)
    else:
        modes = [[2,2],[2,-2]]
        sp, sc = func.generate_signal(sg, delta_f, f_min, approximant, modes=modes)
    s_f = func.signal_to_detector_frame(detect, sp, sc, sg)
    s_f.resize(len(PSD))
    return s_f

def get_sigma_sq_inside_bin(binindex, sg, PSD, approximant, HMs, delta_f, f_min, detect):
    indices_file = "%s.hdf" %binindex
    injInd = read_ind_files(indices_file)
    sigmasq_list = []
    if(len(injInd) != 0):
        for n in range(len(injInd)):    
            inj_ind = injInd[n]

            s_f = get_signal(sg[inj_ind], delta_f, f_min, approximant, HMs, detect)
            overlap = pycbc.filter.matchedfilter.overlap(s_f, s_f, psd=PSD, low_frequency_cutoff=f_min, normalized=False)
            sigmasq = pycbc.filter.matchedfilter.sigmasq(s_f, psd=PSD, low_frequency_cutoff=f_min)
            sigmasq_list.append(sigmasq)
    else:
        print('No signals is bin', binindex)
        sigmasq_list.append([])     
    return sigmasq_list
    
def check_parameter_distribution_inbin(sg, binindex):
    indices_file = "%s.hdf" %binindex
    injInd = read_ind_files(indices_file)
    
    m1 = [sg[k].m1 for k in injInd]
    m2 = [sg[k].m2 for k in injInd]

    plt.hist(m1)
    plt.show

In [16]:
f_min = 30.0
delta_f = 1.0/32
delta_t = 1.0/2048
detect = detector.Detector('H1')
approximant = 'IMRPhenomXPHM'
HMs = True

nsets = 10000
nsignals = 10

nbinsx = 14
nbinsy = 12
Nbins = nbinsx*nbinsy

inj_dir = "/work/rahul.dhurkunde/HM_and_precession/injections/100000_inj/nonaligned_injections/"
inj_type = 'HM_prec'
psd_file = '/work/rahul.dhurkunde/HM_and_precession/psds/ZDHP.txt'

length = int(1.0/(delta_f*delta_t)/2 + 1)
PSD = pycbc.psd.read.from_txt(psd_file, length, delta_f, f_min, is_asd_file=True)

In [17]:
start = time.time()
print('Loading injections -- this may take a while')
sg, sg_m1, sg_m2, mtotal, q = read_injections(nsets, nsignals, f_min, inj_dir)
end = time.time()
print("Inj loading time", end-start)


start = time.time()
print('Loading FFs -- this may take a while')
FF = read_FFs(inj_type, nsets)
end = time.time()
print("FFs loading time", end-start)

Loading injections -- this may take a while
Inj loading time 248.8186273574829
Loading FFs -- this may take a while
FFs loading time 22.555500507354736


In [22]:
srf_file = "../%s_%s.hdf" %(inj_type, nsets)
SRF, xedge, yedge = read_SRF(srf_file)

for n in range(11,12):
    ff_list = get_FF_inside_bin(n, FF)
    sigmasq_list = get_sigma_sq_inside_bin(n, sg, PSD, approximant, HMs, delta_f, f_min, detect)
    manual_srf = compute_SRF_for_bin(n, sg, ff_list, sigmasq_list)
    print(len(ff_list), len(sigmasq_list), 'Manual(d=constant)', manual_srf, 'Old', SRF[n], '\n')
    print(sigmasq_list)

40 40 Manual(d=constant) 0.7787823322981504 Old 0.824677701503891 

[17.22931338902547, 171.17241763708452, 52.23896140297967, 270.4041483916925, 42.03789030553355, 249.04339175132827, 73.28831041123713, 307.8218975996989, 121.96435390644541, 138.67204066299195, 148.99324165867355, 27.73251613359191, 68.61373741901042, 238.146117903957, 81.34682375535189, 72.07554245162454, 126.45458376229274, 82.6224237733815, 70.26160390137213, 74.31367283354331, 112.30716465395801, 61.05367516994274, 506.14739124172775, 89.75304455611318, 209.08231012443295, 110.07259653228276, 43.63754663931867, 209.15015892921778, 26.858149958200492, 40.155006235203096, 9.38678933033151, 68.65205268458794, 142.8963004046532, 30.524602055394904, 44.72375400975148, 117.90197027353986, 528.0801428386129, 144.72050857433166, 17.075779779749734, 11.633491046707737]


In [25]:
srf_file = "../%s_%s.hdf" %(inj_type, nsets)
SRF, xedge, yedge = read_SRF(srf_file)

for n in range(6,12):
    ff_list = get_FF_inside_bin(n, FF)
    tmpsigmasq = get_sigma_sq_inside_bin(n, sg, PSD, approximant, HMs, delta_f, f_min, detect)
    manual_srf = compute_SRF_for_bin(n, sg, ff_list, tmpsigmasq)
    print(len(ff_list), len(tmpsigmasq), 'Manual(d=constant)', manual_srf, 'Old', SRF[n], '\n')

90 90 Manual(d=constant) 0.717301444539793 Old 0.6074165911544505 

71 71 Manual(d=constant) 0.6296815394717521 Old 0.4164695780735702 

69 69 Manual(d=constant) 0.5856013768073222 Old 0.33746802889646016 

76 76 Manual(d=constant) 0.7956976830901281 Old 0.23511613151302022 

64 64 Manual(d=constant) 0.7219310501018098 Old 0.6322443083714937 

40 40 Manual(d=constant) 0.7787823322981504 Old 0.824677701503891 

